In [ ]:
import os
import requests
from msal import ConfidentialClientApplication

In [ ]:
# ── CONFIG ─────────────────────────────────────────────────────────────────────
TENANT_ID       = os.getenv("AZ_TENANT_ID")
CLIENT_ID       = os.getenv("AZ_CLIENT_ID")
CLIENT_SECRET   = os.getenv("AZ_CLIENT_SECRET")
FABRIC_SUB_ID   = os.getenv("AZ_SUBSCRIPTION_ID")

WEST_REGION     = "West US"
EAST_REGION     = "East US"

# Scopes for Fabric REST API
SCOPE           = "https://management.azure.com/.default"
API_VERSION     = "2024-05-01-preview"

In [ ]:
# ── AUTH ────────────────────────────────────────────────────────────────────────
app = ConfidentialClientApplication(
    CLIENT_ID, authority=f"https://login.microsoftonline.com/{TENANT_ID}", client_credential=CLIENT_SECRET
)
token = app.acquire_token_for_client([SCOPE])["access_token"]
HEADERS = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

In [ ]:
# ── STEP 1: LIST WORKSPACES IN WEST US ───────────────────────────────────────────
url_list = (
    f"https://management.azure.com/subscriptions/{FABRIC_SUB_ID}"
    f"/providers/Microsoft.Fabric/workspaces?api-version={API_VERSION}&$filter=location eq '{WEST_REGION}'"
)
resp = requests.get(url_list, headers=HEADERS)
resp.raise_for_status()
workspaces = resp.json()["value"]

In [ ]:
# ── STEP 2: EXPORT ARTIFACTS ────────────────────────────────────────────────────
# For each workspace, export Git-backed content (if enabled) or use export APIs per artifact type.
# Here’s a general pattern—customize per artifact:
for ws in workspaces:
    ws_name = ws["name"]
    resource_group = ws["id"].split("/")[4]
    print(f"Exporting workspace {ws_name}...")
    export_url = (
        f"https://management.azure.com{ws['id']}/export?api-version={API_VERSION}"
    )
    r2 = requests.post(export_url, headers=HEADERS, json={})
    r2.raise_for_status()
    job = r2.json()
    print("  Export job queued:", job["name"])
    # You’d poll until job finishes and download a .zip

In [ ]:
# ── STEP 3: PROVISION EAST US WORKSPACES ─────────────────────────────────────────
for ws in workspaces:
    ws_name = ws["name"]
    rg = ws["id"].split("/")[4]
    print(f"Creating workspace {ws_name}-dr in {EAST_REGION}...")
    create_url = (
        f"https://management.azure.com/subscriptions/{FABRIC_SUB_ID}"
        f"/resourceGroups/{rg}/providers/Microsoft.Fabric/workspaces/{ws_name}-dr"
        f"?api-version={API_VERSION}"
    )
    payload = {
        "location": EAST_REGION,
        "properties": {
            "friendlyName": ws_name + "-dr",
            "capacityName": "<east-capacity-name>",
            # link Git repo if desired...
        }
    }
    r3 = requests.put(create_url, headers=HEADERS, json=payload)
    r3.raise_for_status()
    print(f"  Workspace {ws_name}-dr provisioning started")




In [ ]:
# ── STEP 4: IMPORT ARTIFACTS INTO EAST ───────────────────────────────────────────
# After the DR workspaces exist and exports are downloaded to ./exports:
for ws in workspaces:
    dr_name = ws["name"] + "-dr"
    zip_path = f"./exports/{ws['name']}.zip"
    print(f"Importing into {dr_name} from {zip_path}...")
    import_url = (
        f"https://management.azure.com/subscriptions/{FABRIC_SUB_ID}"
        f"/resourceGroups/{rg}/providers/Microsoft.Fabric/workspaces/{dr_name}/import"
        f"?api-version={API_VERSION}"
    )
    with open(zip_path, "rb") as fh:
        files = {"file": fh}
        r4 = requests.post(import_url, headers={"Authorization":HEADERS["Authorization"]}, files=files)
    r4.raise_for_status()
    print("  Import queued:", r4.json()["name"])